<a href="https://colab.research.google.com/github/Sritam28/AgEnTsIrI/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("/content/IMDB_Prosessed_V1.csv")

In [3]:
data.head()

,Unnamed: 0,review,sentiment
0,0,one reviewer mention watch oz episode youll ho...,positive
1,1,wonderful little production filming technique ...,positive
2,2,think wonderful way spend time hot summer week...,positive
3,3,basically theres family little boy jake think ...,negative
4,4,petter matteis love time money visually stunni...,positive


In [4]:
data.tail()

,Unnamed: 0,review,sentiment
2618,2618,come across movie brothersister relationship i...,positive
2619,2619,many viewer say good movie watch since feel is...,positive
2620,2620,los angeles indie film brat john carpenter fre...,negative
2621,2621,love show watch rerun every day even though se...,positive
2622,2622,drive lesson writer critically acclaim film mr...,NaN


In [5]:
data.columns

Index(['Unnamed: 0', 'review', 'sentiment'], dtype='object')

In [6]:
data.shape

(2623, 3)

In [7]:
data.describe()

,Unnamed: 0
count,2623.000000
mean,1311.000000
std,757.339202
min,0.000000
25%,655.500000
50%,1311.000000
75%,1966.500000
max,2622.000000


In [8]:
data.isnull().sum()

Unnamed: 0    0
review        0
sentiment     1
dtype: int64

In [9]:
data.isnull().any()

Unnamed: 0    False
review        False
sentiment      True
dtype: bool

In [10]:
data['sentiment'].value_counts()

positive    1322
negative    1300
Name: sentiment, dtype: int64

Tokenization of data

In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize

In [12]:
import spacy
import re, string, unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:

#tokenization begins
tokenizers = ToktokTokenizer()
stopwords = nltk.corpus.stopwords.words('english')

In [15]:
def noiseremoval(text):
  soup = BeautifulSoup(text,"html.parser")
  text = soup.get_text()
  text = re.sub('\[[^]]*\]','',text)
  return text
#BeautifulSoup is used to data scrappping

In [16]:
data['review'] = data['review'].apply(noiseremoval)

In [17]:
data.head()

,Unnamed: 0,review,sentiment
0,0,one reviewer mention watch oz episode youll ho...,positive
1,1,wonderful little production filming technique ...,positive
2,2,think wonderful way spend time hot summer week...,positive
3,3,basically theres family little boy jake think ...,negative
4,4,petter matteis love time money visually stunni...,positive


In [18]:

#STEMMING
def stemmer(text):
  ps = nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text

In [19]:
data['review']=data['review'].apply(stemmer)

In [20]:
data.head()

,Unnamed: 0,review,sentiment
0,0,one review mention watch oz episod youll hook ...,positive
1,1,wonder littl product film techniqu unassum old...,positive
2,2,think wonder way spend time hot summer weekend...,positive
3,3,basic there famili littl boy jake think there ...,negative
4,4,petter mattei love time money visual stun film...,positive


In [21]:
#Remove stopwords
def remove_stopwords(text,is_lower_case=False):
  tokenizers = ToktokTokenizer()
  tokens = tokenizers.tokenize(text)
  tokens = [i.strip() for i in tokens]
  if is_lower_case:
    filtokens = [i for i in tokens if tokens not in stopwords]
  else:
    filtokens = [i for i in tokens if i.lower() not in stopwords]
  filtered_texts = ' '.join(filtokens)
  return filtered_texts

In [22]:
data['review'] = data['review'].apply(remove_stopwords)

In [23]:
data.head()

,Unnamed: 0,review,sentiment
0,0,one review mention watch oz episod youll hook ...,positive
1,1,wonder littl product film techniqu unassum old...,positive
2,2,think wonder way spend time hot summer weekend...,positive
3,3,basic famili littl boy jake think zombi closet...,negative
4,4,petter mattei love time money visual stun film...,positive


In [24]:
#split the dataset
#train dataset
train_reviews_data = data.review[:2000]

In [25]:
#test data set
test_reviews_data=data.review[2000:]

In [26]:
#Bag of words technique
cv = CountVectorizer(min_df = 0,max_df=1,binary=False,ngram_range=(1,3))
cv_train = cv.fit_transform(train_reviews_data)
cv_test = cv.transform(test_reviews_data)
print('BOW_cv_train:',cv_train.shape)
print('BOW_cv_test:',cv_test.shape)


BOW_cv_train: (2000, 393803)
BOW_cv_test: (623, 393803)


In [27]:
#Tfidf vectorizer
tf = TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
tf_train = tf.fit_transform(train_reviews_data)
tf_test = tf.transform(test_reviews_data)
print('Tfidf_train:',tf_train.shape)
print('Tfidf_test: ',tf_test.shape)

Tfidf_train: (2000, 393803)
Tfidf_test:  (623, 393803)


Label Encoding

In [28]:
#labelling the sentiment data
label = LabelBinarizer()
sentiment_data = label.fit_transform(data['review'])
print(sentiment_data.shape)

(2623, 2623)


In [29]:
sentiment_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [30]:
train_data = data.sentiment[:2000]
test_data = data.sentiment[2000:]

In [31]:
#logistic regression model
logistic = LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr_bow =logistic.fit(cv_train,train_data)
print(lr_bow)

LogisticRegression(C=1, max_iter=500, random_state=42)


In [32]:
#Predicting the model for bag of words
bow_predict = logistic.predict(cv_test)
print(bow_predict)


['negative' 'positive' 'positive' 'positive' 'positive' 'negative'
 'positive' 'positive' 'positive' 'negative' 'negative' 'positive'
 'negative' 'negative' 'positive' 'positive' 'positive' 'positive'
 'positive' 'negative' 'negative' 'negative' 'negative' 'positive'
 'positive' 'negative' 'positive' 'positive' 'positive' 'negative'
 'negative' 'negative' 'negative' 'negative' 'negative' 'negative'
 'positive' 'positive' 'positive' 'positive' 'positive' 'negative'
 'negative' 'positive' 'negative' 'negative' 'positive' 'positive'
 'positive' 'positive' 'negative' 'positive' 'negative' 'negative'
 'positive' 'positive' 'positive' 'negative' 'positive' 'positive'
 'positive' 'negative' 'negative' 'positive' 'positive' 'positive'
 'positive' 'negative' 'positive' 'positive' 'positive' 'negative'
 'negative' 'positive' 'negative' 'positive' 'positive' 'positive'
 'positive' 'negative' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'negative' 'negative' 'positive' 'posit

In [35]:
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf_predict)

['positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'negative' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'negative' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'negative' 'positive' 'positive' 'negative'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'negative'
 'positive' 'positive' 'positive' 'negative' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'negative'
 'positive' 'positive' 'negative' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'positive'
 'positive' 'positive' 'positive' 'positive' 'positive' 'posit